In [150]:
import pandas as pd 

In [151]:
# Load data files
df_survey = pd.read_excel('data/private_dataL.xlsx', sheet_name='Sheet 1')
df_results = pd.read_excel('data/public_data_resultsL.xlsx', sheet_name='Sheet 1')



In [152]:
import pandas as pd

survey_counts = pd.crosstab(
    df_survey['evote'].map({0: 'Offline', 1: 'Online'}),
    df_survey['party']
)

# Premenuj index a resetni
survey_counts.index.name = 'party_type'
survey_counts = survey_counts.reset_index()

# remove party row name party
survey_counts.columns.name = None
# remove

survey_counts

,party_type,Green,Invalid vote,Red
0,Offline,86,3,48
1,Online,41,0,22


In [154]:
# select Polling stations into 1 row and sum them up
offline = df_results[df_results["party"].str.startswith("Polling station")]
offline_sum = offline[["Red", "Green", "Invalid ballots"]].sum()

# Create new row offline
offline_row = pd.DataFrame([{
    "party": "offline KOKOT",
    **offline_sum.to_dict()
}])

# get others
rest = df_results[~df_results["party"].str.startswith("Polling station")]
# join 
df_results_joined = pd.concat([rest, offline_row], ignore_index=True)

df_results_joined = df_final.sort_values("party").reset_index(drop=True)


df_final

,party,Red,Green,Invalid ballots
0,E-votes,130,206,1
1,Offline votes,278,406,18
